In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tslearn.metrics import dtw

In [2]:
data_small = pd.read_csv('3months_under10_000.csv',index_col = 'day', parse_dates = True)

C:\Users\nakae\AppData\Local\Temp\ipykernel_15136\4100203135.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  data_small = pd.read_csv('3months_under10_000.csv',index_col = 'day', parse_dates = True)


In [3]:
data_small.head()

,shieldid,dns_qname,dns_qdomainname,num_clients,num_responses,timefirstseen,timelastseen,anomaly,shieldid_count,rank,domain,created,updated,expires,whois
day,,,,,,,,,,,,,,,
2023-01-04,enilokmacy,1334519-19.chat.api.drift.com,drift.com,1,16,2023-01-04 00:15:40.307,2023-01-04 00:15:40.307,0,13,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au
2023-01-10,enilokmacy,istio.api.drift.com,drift.com,1,24,2023-01-10 13:36:20.757,2023-01-10 13:36:20.757,0,19,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au
2023-01-09,fecjavfec,1500556-6.chat.api.drift.com,drift.com,1,8,2023-01-09 15:00:47.243,2023-01-09 15:00:47.243,0,15,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au
2023-01-20,fecjavfec,metrics.api.drift.com,drift.com,4,260,2023-01-20 10:08:52.388,2023-01-20 20:55:08.464,0,20,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au
2023-01-20,oreethecaw,presence.api.drift.com,drift.com,15,140,2023-01-20 02:55:21.611,2023-01-20 14:29:37.833,0,20,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au


In [4]:
data_small.whois.fillna('unknown',inplace=True)

In [5]:
data_small['timelastseen'] = pd.to_datetime(data_small['timelastseen'])

In [6]:
data_small['timefirstseen'] = pd.to_datetime(data_small['timefirstseen'])

In [7]:
data_small['timeseen'] = data_small['timelastseen']-data_small['timefirstseen']

In [8]:
#organize data like in linear regression example
counts = data_small.pivot_table(values = 'num_clients',index = data_small.index.date,
                         columns = 'dns_qdomainname',
                         aggfunc=sum,
                         fill_value = 0)

In [9]:
#normalize data
counts.max().sort_values(ascending=False).to_csv('counts.csv')
counts = counts/counts.max()

In [10]:
#create domians by checking similairty to 2 anomalies
domains = []
dns = data_small.dns_qdomainname.unique()
similarity_slider = .6
#numba this
for i in range (len(dns)-1):
    val = dtw(counts['beside.media'],counts[dns[i]])
    if (val <= similarity_slider) & (dtw(counts['123w0w.com'],counts[dns[i]]) <= similarity_slider):
        domains.append(dns[i])

In [11]:
len(domains)

1161

In [12]:
data_small.dns_qdomainname.nunique()

168877

In [13]:
domains

['servogram.io',
 'band.us',
 'calia.com',
 'wcoanimedub.tv',
 'armakeup.com',
 'safelifedefense.com',
 'boardgamegeek.com',
 'avail.co',
 'sbrchamber.com',
 'nkpmedical.com',
 'babbel.com',
 'needtoknowfacts.com',
 'forfamilymall.com',
 'andimejia.com',
 'magnus.ms',
 'ttlstatic.com',
 'lh.social',
 'roundme.com',
 'history.com',
 'as.me',
 'bizorgraphics.com',
 'bbcselect.com',
 'beautiful.ai',
 'danmarketplace.com',
 'amcplus.com',
 'smallseotools.com',
 'tango.us',
 'cole.ca',
 'pngjoy.com',
 'encentivizer.com',
 'unnumelom.com',
 'iott.cc',
 'aglowlifestyle.com',
 'undark.org',
 'jar-download.com',
 'donjohnston.net',
 'mmsend50.com',
 'proquest.com',
 'segreencolumn.com',
 'open.ai',
 'wsktbf.net',
 'manilarepublic.com',
 'ticketschool.com',
 'morriscenter.org',
 'lbcgroup.tv',
 'itworkscdn.net',
 'adoptopenjdk.net',
 'loadcomplete.com',
 'them.us',
 'updpopcorntime.xyz',
 'raab3frog.com',
 'yale.edu',
 'static-carhp.com',
 'dtra.mil',
 'dakasapps.com',
 'kidsempoweredonthemove.o

In [14]:

data_small.loc[:,'anomaly_2'] = 0
data_small.loc[data_small.dns_qdomainname.isin(domains),'anomaly_2'] = 1

In [15]:
data_small

,shieldid,dns_qname,dns_qdomainname,num_clients,num_responses,timefirstseen,timelastseen,anomaly,shieldid_count,rank,domain,created,updated,expires,whois,timeseen,anomaly_2
day,,,,,,,,,,,,,,,,,
2023-01-04,enilokmacy,1334519-19.chat.api.drift.com,drift.com,1,16,2023-01-04 00:15:40.307,2023-01-04 00:15:40.307,0,13,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au,0 days 00:00:00,0
2023-01-10,enilokmacy,istio.api.drift.com,drift.com,1,24,2023-01-10 13:36:20.757,2023-01-10 13:36:20.757,0,19,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au,0 days 00:00:00,0
2023-01-09,fecjavfec,1500556-6.chat.api.drift.com,drift.com,1,8,2023-01-09 15:00:47.243,2023-01-09 15:00:47.243,0,15,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au,0 days 00:00:00,0
2023-01-20,fecjavfec,metrics.api.drift.com,drift.com,4,260,2023-01-20 10:08:52.388,2023-01-20 20:55:08.464,0,20,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au,0 days 10:46:16.076000,0
2023-01-20,oreethecaw,presence.api.drift.com,drift.com,15,140,2023-01-20 02:55:21.611,2023-01-20 14:29:37.833,0,20,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au,0 days 11:34:16.222000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-21,thwijijod,cdn-prod-sg.yepgarage.info,yepgarage.info,1,20,2022-12-21 05:28:15.901,2022-12-21 05:28:15.901,0,1,0.0,yepgarage.info,2020-11-19,2020-11-19,2021-11-19,whois.gandi.net,0 days 00:00:00,0
2023-01-12,thwijijod,www.omaemirates.com,omaemirates.com,1,1,2023-01-12 06:01:10.576,2023-01-12 06:01:10.576,0,1,0.0,omaemirates.com,1998-10-28,1998-10-28,2020-10-27,whois.networksolutions.com,0 days 00:00:00,0
2023-01-03,volcagbuca,www.copenhagen.com,copenhagen.com,2,2,2023-01-03 01:18:13.956,2023-01-03 01:18:13.983,0,1,0.0,copenhagen.com,1996-04-02,1996-04-02,2026-04-02,whois.godaddy.com,0 days 00:00:00.027000,0


In [16]:
# tld column
data_small['tld'] = data_small.dns_qdomainname.str.split('.').str[1]

In [17]:
# column of tlds cross referenced with a list of the top tlds used in malicious activity
mal_tlds = ['live','monster','okinawa','top','boutique','beauty','cn','zone','autos','zw','bd','ke','am','sbs','date','pw','quest','cd','bid','cyou','ml','ws','icu','stream','email','uno','cam','xyz','tokyo','help','casa','rest','win']
data_small.loc[:,'mal_tld']=0
data_small.loc[data_small.tld.isin(mal_tlds),'mal_tld']=1
data_small.mal_tld.sum()

36736

In [43]:
data_small.head()

,shieldid,dns_qname,dns_qdomainname,num_clients,num_responses,timefirstseen,timelastseen,anomaly,shieldid_count,rank,domain,created,updated,expires,whois,timeseen,anomaly_2,tld,mal_tld
day,,,,,,,,,,,,,,,,,,,
2023-01-04,enilokmacy,1334519-19.chat.api.drift.com,drift.com,1,16,2023-01-04 00:15:40.307,2023-01-04 00:15:40.307,0,13,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au,0 days 00:00:00,0,com,0
2023-01-10,enilokmacy,istio.api.drift.com,drift.com,1,24,2023-01-10 13:36:20.757,2023-01-10 13:36:20.757,0,19,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au,0 days 00:00:00,0,com,0
2023-01-09,fecjavfec,1500556-6.chat.api.drift.com,drift.com,1,8,2023-01-09 15:00:47.243,2023-01-09 15:00:47.243,0,15,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au,0 days 00:00:00,0,com,0
2023-01-20,fecjavfec,metrics.api.drift.com,drift.com,4,260,2023-01-20 10:08:52.388,2023-01-20 20:55:08.464,0,20,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au,0 days 10:46:16.076000,0,com,0
2023-01-20,oreethecaw,presence.api.drift.com,drift.com,15,140,2023-01-20 02:55:21.611,2023-01-20 14:29:37.833,0,20,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au,0 days 11:34:16.222000,0,com,0


In [18]:
mal_tlds

['live',
 'monster',
 'okinawa',
 'top',
 'boutique',
 'beauty',
 'cn',
 'zone',
 'autos',
 'zw',
 'bd',
 'ke',
 'am',
 'sbs',
 'date',
 'pw',
 'quest',
 'cd',
 'bid',
 'cyou',
 'ml',
 'ws',
 'icu',
 'stream',
 'email',
 'uno',
 'cam',
 'xyz',
 'tokyo',
 'help',
 'casa',
 'rest',
 'win']

In [19]:
# of the potentialy anomalys using anomaly finder 2, how many fall into the list of malicious tld
domains
domains_tld=[i.split('.')[1] for i in domains]
domains_mal = set(domains_tld) & set(mal_tlds)
domains_mal

{'cn', 'email', 'live', 'top', 'xyz'}

In [20]:
# anomaly finder 1?
anomaly1 = data_small.loc[data_small.anomaly==1].dns_qdomainname.unique().tolist()
anomaly1_tld = [i.split('.')[1] for i in domains]
domains1_mal = set(anomaly1_tld) & set(mal_tlds)
domains1_mal

{'cn', 'email', 'live', 'top', 'xyz'}

## Random Forest Classifier

In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, OrdinalEncoder
from sklearn.impute import SimpleImputer

In [23]:
data_small.head()

,shieldid,dns_qname,dns_qdomainname,num_clients,num_responses,timefirstseen,timelastseen,anomaly,shieldid_count,rank,domain,created,updated,expires,whois,timeseen,anomaly_2,tld,mal_tld
day,,,,,,,,,,,,,,,,,,,
2023-01-04,enilokmacy,1334519-19.chat.api.drift.com,drift.com,1,16,2023-01-04 00:15:40.307,2023-01-04 00:15:40.307,0,13,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au,0 days 00:00:00,0,com,0
2023-01-10,enilokmacy,istio.api.drift.com,drift.com,1,24,2023-01-10 13:36:20.757,2023-01-10 13:36:20.757,0,19,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au,0 days 00:00:00,0,com,0
2023-01-09,fecjavfec,1500556-6.chat.api.drift.com,drift.com,1,8,2023-01-09 15:00:47.243,2023-01-09 15:00:47.243,0,15,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au,0 days 00:00:00,0,com,0
2023-01-20,fecjavfec,metrics.api.drift.com,drift.com,4,260,2023-01-20 10:08:52.388,2023-01-20 20:55:08.464,0,20,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au,0 days 10:46:16.076000,0,com,0
2023-01-20,oreethecaw,presence.api.drift.com,drift.com,15,140,2023-01-20 02:55:21.611,2023-01-20 14:29:37.833,0,20,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au,0 days 11:34:16.222000,0,com,0


In [24]:
X = data_small[['num_clients', 'num_responses','shieldid_count','rank']]
#X = data_small.drop(['anomaly','anomaly_2'],axis=1)
y = data_small['anomaly_2']

In [25]:
X.columns

Index(['num_clients', 'num_responses', 'shieldid_count', 'rank'], dtype='object')

In [26]:
numerical_feats = ['num_clients', 'num_responses','shieldid_count','rank']
#ordinal_feats = ['timefirstseen','timelastseen','created','updated','expires','timeseen']
#categorical_feats = ['shieldid','dns_qname','dns_qdomainname','domain','whois']

In [27]:
num_processor = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# cat_processor = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy = 'constant')),
#     ('encoder', OneHotEncoder())   
# ])

# ord_processor = Pipeline(steps=[
#     ('ord_imputer', SimpleImputer(strategy='constant')),
#     ('ord_encoder', OrdinalEncoder())
# ])

In [28]:
feat_processor = ColumnTransformer(transformers = [
    ('num_pipe', num_processor,numerical_feats)
    #('cat_pipe', cat_processor, categorical_feats)
    #('ord_pipe', ord_processor, ordinal_feats)
],remainder='drop')

In [29]:
RF= RandomForestClassifier()

In [30]:
pipe = Pipeline(steps=[
    ('feature_processor', feat_processor),
    ('clf', RF)
])
pipe

Pipeline(steps=[('feature_processor',
                 ColumnTransformer(transformers=[('num_pipe',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['num_clients',
                                                   'num_responses',
                                                   'shieldid_count',
                                                   'rank'])])),
                ('clf', RandomForestClassifier())])

In [31]:
pipe.fit(X,y)

Pipeline(steps=[('feature_processor',
                 ColumnTransformer(transformers=[('num_pipe',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['num_clients',
                                                   'num_responses',
                                                   'shieldid_count',
                                                   'rank'])])),
                ('clf', RandomForestClassifier())])

In [41]:
param_dic = {'clf__min_samples_leaf':[1,4,8], 
          'clf__min_samples_split':[2,4,8], 
          'clf__n_estimators': [10,25,50],#,100], 
          'clf__max_depth':[2,5,10]
          #'class_weight': ['balanced']
         }
grid = GridSearchCV(pipe, 
                    param_dic,
                    cv=5,
                    scoring='precision',
                    n_jobs = -1,
                    verbose=True)


In [42]:
grid.fit(X,y)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


KeyboardInterrupt: 